In [545]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [546]:
#Set directory for input files
import_file_path = r"C:\Users\Selim Elbadri\Dropbox\KLEMS_productivity\Input"
export_file_path = r"C:\Users\Selim Elbadri\Dropbox\KLEMS_productivity\Output"

# 2. Functions & Industry Aggregations

In [547]:
#Function to compute log-diff and drop original variable
def dlog(df, var, group_col='indnum', time_col='yr', base_year=None):
    df = df.copy()
    df = df.sort_values([group_col, time_col])
    newvar = f'dlog_{var}'.replace(' ', '_')

    #Compute log difference
    df[newvar] = df.groupby(group_col)[var].transform(lambda x: np.log(x).diff())

    #Force NaN for base_year or first available year in each group
    if base_year is not None:
        df.loc[df[time_col] == base_year, newvar] = np.nan
    else:
        min_years = df.groupby(group_col)[time_col].transform('min')
        df.loc[df[time_col] == min_years, newvar] = np.nan

    #Drop original variable
    return df.drop(columns=[var])

In [548]:
#Industry aggregations for 1947-1963
aggregate_groups = {
    2936: list(range(29, 37)),
    3740: list(range(37, 41)),
    4144: list(range(41, 45)),
    4749: list(range(47, 50)),
    5152: list(range(51, 53)),
    5456: list(range(54, 57)),
    5758: list(range(57, 59))}

# 3. Cleaning BEA-BLS Experimental Dataset

In [ ]:
#Identify needed variables
experimental_vars = ['yr','indnum','goqi.','iiqi.','vlcol.','vln.','vkit.','vksoft.','vkRD.',
    'vkart.','vkoth.','qkit.','qks.','qkrd.','qka.','qko.','hrs','qlindexcol_merge.', 'qlindexn_merge.']

#Extract datasheets from BEA-BLS Integrated Industry-Level Production Account (Eldridge et al., 2020)
df_experimental_1947to1963 = pd.read_excel(os.path.join(import_file_path, 'industry-production-account-experimental.xlsx'), 
    sheet_name='1947-1963', skiprows=1, usecols=experimental_vars)
df_experimental_1963to2016 = pd.read_excel(os.path.join(import_file_path, 'industry-production-account-experimental.xlsx'),
    sheet_name='1963-2016', skiprows=1, usecols=experimental_vars)

#Quantity indices to be log-differenced
q_indices = ['goqi.','iiqi.','qkit.','qks.','qkrd.','qka.','qko.','qlindexcol_merge.','qlindexn_merge.','hrs']

#Generate log-difference for quantity indices in both sheets
for v in q_indices:
    df_experimental_1947to1963 = dlog(df_experimental_1947to1963, v, base_year=1947)

for v in q_indices:
    df_experimental_1963to2016 = dlog(df_experimental_1963to2016, v, base_year=1963)

In [ ]:
#Restrict BEA-BLS Experimental to 1997 (for nominal variables) and to 1998 (for growth variables)
nom_var     = ['vlcol.','vln.', 'vkit.', 'vksoft.', 'vkRD.', 'vkart.', 'vkoth.']
growth_var  = ['dlog_goqi.', 'dlog_iiqi.', 'dlog_qkit.', 'dlog_qks.', 'dlog_qkrd.', 
                   'dlog_qka.', 'dlog_qko.', 'dlog_qlindexcol_merge.', 'dlog_qlindexn_merge.', 'dlog_hrs']

#Nominal variables stop after 1996
df_experimental_1963to2016 = df_experimental_1963to2016.copy()
for v in nom_var:
    df_experimental_1963to2016.loc[(df_experimental_1963to2016['yr'] < 1963) | (df_experimental_1963to2016['yr'] > 1996),v] = np.nan

#Growth variables stop after 1997
for v in growth_var:
    df_experimental_1963to2016.loc[(df_experimental_1963to2016['yr'] < 1963) | (df_experimental_1963to2016['yr'] > 1997),v] = np.nan

# 4. Cleaning WK2017 Data

In [ ]:
#Identify needed variables
klems_vars = ['year', 'industry', 'gross output', 'capital', 'labor', 'intermediate']

#Extract the required datasheets from US KLEMS, March 2017 (Jorgenson et al., 2017)
df_klems = pd.read_excel(os.path.join(import_file_path, 'usa_wk_mar_2017.xlsx'), sheet_name='KLEMdata', skiprows=1, usecols=klems_vars)

#Rename panel identifiers to be consistent with df_experimental
df_klems.rename(columns={'industry': 'indnum','year': 'yr'}, inplace=True)

#Rename variables to be consistent with KLEMS literature
df_klems.rename(columns={'gross output': 'GO','capital': 'CAP','labor': 'LAB','intermediate': 'II'}, inplace=True)

In [ ]:
#For consistency with other datasets, consolidate federal government and state & local government from 2 industries each to 1 industry each
federal_inds          = [62, 63]
state_local_inds      = [64, 65]

#Sum nominal variables for indnum 62/63
federal               = df_klems[df_klems['indnum'].isin(federal_inds)].groupby('yr', as_index=False)[['GO', 'CAP', 'LAB', 'II']].sum()

#Sum nominal variables for indnum 64/65
state_local           = df_klems[df_klems['indnum'].isin(state_local_inds)].groupby('yr', as_index=False)[['GO', 'CAP', 'LAB', 'II']].sum()

federal['indnum']     = 62           #Consolidated Federal government indnum = 62
state_local['indnum'] = 63           #Consolidated state & local indnum = 63

#Remove the original rows and append new rows
df_klems              = df_klems[~df_klems['indnum'].isin(federal_inds + state_local_inds)]
df_klems              = pd.concat([df_klems, federal, state_local], ignore_index=True)

#Sort by year and indnum
df_klems              = df_klems.sort_values(['indnum','yr']).reset_index(drop=True)

In [554]:
#Build panel for broad industries starting from 1947-
df_klems_1947to2014 = (
    df_klems.assign(indnum=df_klems['indnum'].replace({i:new for new, olds in aggregate_groups.items() for i in olds}))
    .groupby(['yr','indnum'], as_index=False)[['GO', 'CAP', 'LAB', 'II']].sum())

#Build panel for finer industries starting 1963-
df_klems_1963to2014 = df_klems.loc[df_klems['yr'] >= 1963].copy()

# 5. Cleaning BEA-BLS Capital Dataset

In [ ]:
#Name of sheets to extract data from
capital_sheets = [
    'Capital_Art_Quantity','Capital_R&D_Quantity','Capital_IT_Quantity','Capital_Other_Quantity','Capital_Software_Quantity',
    'Capital_Art Compensation', 'Capital_R&D Compensation','Capital_IT Compensation','Capital_Other Compensation','Capital_Software Compensation',
    'Labor_Col_Quantity','Labor_NoCol_Quantity','Labor_Col Compensation','Labor_NoCol Compensation',
    'Energy_Quantity','Materials_Quantity','Services_Quantity','Energy Compensation', 
    'Materials Compensation','Service Compensation','Gross Output', 'Gross Output_Quantity', 'Labor Hours_Quantity'
]

#Extract industry-level from BEA-BLS Integrated Industry-Level Production Account (Eldridge et al., 2025)
long_data = []
for sheet in capital_sheets:
    df_tmp = pd.read_excel(os.path.join(import_file_path, 'industry-production-account-capital.xlsx'), sheet_name=sheet, header=1).dropna(how='all')
    df_tmp = df_tmp.rename(columns={df_tmp.columns[0]: 'industry_description'})
    df_tmp = df_tmp.melt(id_vars='industry_description', var_name='year', value_name=sheet)
    long_data.append(df_tmp)
df_capital_1997to2023 = reduce(lambda l, r: pd.merge(l, r, on=['industry_description','year'], how='outer'), long_data)
df_capital_1997to2023 = df_capital_1997to2023.rename(columns={'year':'yr','industry_description':'Description'})

#Create a sequential `indnum` for each unique Description with existing order
order                           = df_capital_1997to2023['Description'].drop_duplicates().tolist()
mapping                         = {desc: i+1 for i, desc in enumerate(order)}
df_capital_1997to2023['indnum'] = df_capital_1997to2023['Description'].map(mapping).astype('Int64')
df_capital_1997to2023['yr']     = pd.to_numeric(df_capital_1997to2023['yr'], errors='coerce')
df_capital_1997to2023           = df_capital_1997to2023.drop(columns='Description')

#Move panel identifiers first
cols                            = ['indnum', 'yr'] + [c for c in df_capital_1997to2023.columns if c not in ['indnum', 'yr']]
df_capital_1997to2023           = df_capital_1997to2023[cols]
df_capital_1997to2023           = df_capital_1997to2023.sort_values(['indnum', 'yr']).reset_index(drop=True)

In [ ]:
#Rename df_capital_1997to2023 variables to match df_experimental variable names
experimental_dictionary = {
    'Gross Output': 'go.','Capital_IT Compensation': 'vkit.','Capital_Software Compensation': 'vksoft.',
    'Capital_R&D Compensation': 'vkRD.','Capital_Art Compensation': 'vkart.','Capital_Other Compensation': 'vkoth.',
    'Labor_Col Compensation': 'vlcol.','Labor_NoCol Compensation': 'vln.','Gross Output_Quantity': 'goqi.',
    'Capital_IT_Quantity': 'qkit.','Capital_Software_Quantity': 'qks.','Capital_R&D_Quantity': 'qkrd.',
    'Capital_Art_Quantity': 'qka.','Capital_Other_Quantity': 'qko.','Labor_Col_Quantity': 'qlindexcol_merge.',
    'Labor_NoCol_Quantity': 'qlindexn_merge.','Labor Hours_Quantity': 'hrs','Energy_Quantity': 'qien.',
    'Materials_Quantity': 'qimt.','Services_Quantity': 'qisv.','Service Compensation': 'visv.',
    'Materials Compensation': 'vimt.','Energy Compensation': 'vien.'
}

#Rename variables using experimental_dictionary
rename_dict           = {k: v for k, v in experimental_dictionary.items() if k in df_capital_1997to2023.columns}
df_capital_1997to2023 = df_capital_1997to2023.rename(columns=rename_dict)

#Log difference the variables of interest
q_indices             = ['goqi.','qkit.','qks.','qkrd.','qka.','qko.','qlindexcol_merge.','qlindexn_merge.','hrs','qien.','qimt.','qisv.']
for v in q_indices:
    if v in df_capital_1997to2023.columns:
        df_capital_1997to2023 = dlog(df_capital_1997to2023, v)

In [ ]:
#Use BEA-BLS Capital to compute nominal variables GO, II, LAB, CAP (needed for 2015-, both for broad industries and finer industries)
df_capital_nominal = (df_capital_1997to2023[df_capital_1997to2023['yr'] >= 2015].copy())

nominal_agg_map = {
    'GO' : ['go.'],
    'CAP': ['vkit.', 'vksoft.', 'vkRD.', 'vkart.', 'vkoth.'],
    'LAB': ['vln.', 'vlcol.'],
    'II' : ['visv.', 'vimt.', 'vien.']
}
for newvar, cols in nominal_agg_map.items():
    cols_present               = [c for c in cols if c in df_capital_nominal.columns]
    df_capital_nominal[newvar] = df_capital_nominal[cols_present].apply(pd.to_numeric, errors='coerce').sum(axis=1, min_count=1)

#Keep nominal variables before 2015 missing for 1963 industries
df_capital_nominal_start1963 = df_capital_nominal[['yr','indnum','GO','II','LAB','CAP']].reset_index(drop=True)

#Aggregate some industries to match 1947-1963 
df_capital_nominal_start1947 = (
    df_capital_nominal_start1963.copy()
    .assign(indnum=lambda d: d['indnum'].map({i:new for new, old in aggregate_groups.items() for i in old}).fillna(d['indnum']).astype('Int64'))
    .groupby(['yr','indnum'], as_index=False)[['GO','II','LAB','CAP']].sum())
df_capital_nominal_start1947 = df_capital_nominal_start1947.sort_values(by=['indnum', 'yr']).reset_index(drop=True)

In [ ]:
## -- Summary 
##In the cleaning stage, we have prepared 7 relevant dataframes:
#df_experimental_1963to2016   : Quantity indices and compensation for factor components (1963-2016)
#df_capital_1997to2023        : Quantity indices and compensation for factor components (1997-2023)
#df_klems_1963to2014          : Nominal GO, II, CAP, LAB for 1963 industry aggregations (1963-2014)
#df_capital_nominal_start1963 : Nominal GO, II, CAP, LAB for 1963 industry aggregations (2015-2023)

#df_klems_1947to2014          : Nominal GO, II, CAP, LAB for 1947 industry aggregations (1947-2014)
#df_capital_nominal_start1947 : Nominal GO, II, CAP, LAB for 1947 industry aggregations (2015-2023)
#df_experimental_1947to1963   : Quantity indices and compensation for factor components (1947-1963)

# 6. Merging the Datasets for 1963-2023

In [539]:
#Append df_klems_1963to2014 & df_capital_nominal_start1963 to create 1963-2023 GO, II, CAP, LAB panel
df_nom_1963to2023 = pd.concat([df_klems_1963to2014, df_capital_nominal_start1963], ignore_index=True)
df_nom_1963to2023 = df_nom_1963to2023.sort_values(by=['indnum', 'yr']).reset_index(drop=True)

In [ ]:
#Merging quantity indices and compensation for factor components (1963-1996/97 & 1997/98-2023)  
all_cols                   = list(set(df_experimental_1963to2016.columns).union(df_capital_1997to2023.columns))
df_experimental_1963to2016 = df_experimental_1963to2016.reindex(columns=all_cols)
df_capital_1997to2023      = df_capital_1997to2023.reindex(columns=all_cols)

#Merge both datasets together
df_extended = pd.merge(df_experimental_1963to2016,df_capital_1997to2023,on=["indnum", "yr"], how="outer", suffixes=("_exp", "_cap"))

#If same column appears twice, then coalesce
for col in all_cols:
    if col + "_exp" in df_extended and col + "_cap" in df_extended:
        df_extended[col] = df_extended[col + "_exp"].combine_first(df_extended[col + "_cap"])
        df_extended      = df_extended.drop(columns=[col + "_exp", col + "_cap"])

#Sort by indnum-yr
df_extended          = df_extended.sort_values(by=["indnum", "yr"]).reset_index(drop=True)

#Reorder variables after merging
cols                 = df_extended.columns.tolist()
merge_key            = ["indnum", "yr"]
v_vars               = [c for c in cols if c.startswith("v") and c not in merge_key]
dlog_vars            = [c for c in cols if c.startswith("dlog_")]
interm_inp           = ["dlog_iiqi.", "vien.", "vimt.", "visv.","dlog_qisv.", "dlog_qimt.", "dlog_qien."]   #Keep II in the end because they are represented by different variables in both datasets
               
#Exclude interm_inp from v_vars and dlog_vars to avoid duplicates
v_vars               = [c for c in v_vars if c not in interm_inp]
dlog_vars            = [c for c in dlog_vars if c not in interm_inp]

# Construct final order
final_order          = merge_key + v_vars + dlog_vars + interm_inp
df_qindices_63to2023 = df_extended[final_order]

In [541]:
#Merge panel data with nominal values and panel data with quantity indices and compensation for factor components
df_1963to2023 = pd.merge(df_nom_1963to2023,df_qindices_63to2023,on=['indnum', 'yr'],how='inner') 
df_1963to2023 = df_1963to2023.sort_values(by=['indnum', 'yr']).reset_index(drop=True)
df_1963to2023 = df_1963to2023.sort_values(by=['indnum', 'yr']).reset_index(drop=True)
other_cols    = [c for c in df_1963to2023.columns if c not in ["indnum", "yr"]]
df_1963to2023 = df_1963to2023[["indnum", "yr"] + other_cols]


# 7. Merging the Datasets for 1947-1963

In [ ]:
#Create a panel dataset for nominal values 1947-1963 (merging KLEMS (1947-2014) and BEA-BLS Capital (2015-2023))
all_cols                     = list(set(df_klems_1947to2014.columns).union(df_capital_nominal_start1947.columns))
df_klems_1947to2014          = df_klems_1947to2014.reindex(columns=all_cols)
df_capital_nominal_start1947 = df_capital_nominal_start1947.reindex(columns=all_cols)

#Append both datasets
df_nom_1947to2023            = pd.concat([df_klems_1947to2014, df_capital_nominal_start1947], ignore_index=True)

#Sort by panel identifiers (indnum, yr)
df_nom_1947to2023            = df_nom_1947to2023.sort_values(by=['indnum', 'yr']).reset_index(drop=True)
other_cols                   = [c for c in df_nom_1947to2023.columns if c not in ["indnum", "yr"]]
df_nom_1947to2023            = df_nom_1947to2023[["indnum", "yr"] + other_cols]

In [ ]:
#Merge df_nom_1947to2023 with df_experimental_1947to1963
df_1947to2023   = pd.merge(df_nom_1947to2023,df_experimental_1947to1963,on=["indnum", "yr"],how="outer",suffixes=("_exp63", "_full"))  # keeps all rows from both
df_1947to2023   = df_1947to2023.sort_values(by=["indnum", "yr"]).reset_index(drop=True)
other_cols      = [c for c in df_1947to2023.columns if c not in ["indnum", "yr"]]
df_1947to2023   = df_1947to2023[["indnum", "yr"] + other_cols]

# 8. Combine 1947-2023 and 1963-2023 Datasets and Export

In [ ]:
#Create a variable definition DataFrame (only Variable + Description)
var_defs = pd.DataFrame({
    "Variable": [
        "indnum", "yr", "GO", "CAP", "LAB", "II", "vkit.",
        "vln.", "vkRD.", "vkoth.", "vlcol.", "vkart.", "vksoft.", "dlog_qko.",
        "dlog_qkit.", "dlog_qlindexcol_merge.", "dlog_qkrd.", "dlog_goqi.",
        "dlog_qlindexn_merge.", "dlog_qka.", "dlog_hrs", "dlog_qks.", "dlog_iiqi.",
        "vien.", "vimt.", "visv.", "dlog_qisv.", "dlog_qimt.", "dlog_qien."
    ],
    "Description": [
        "Industry identifier",                                   # indnum
        "Year",                                                  # yr
        "Nominal Gross Output",                                  # GO
        "Nominal Capital Compensation",                          # CAP
        "Nominal Labor Compensation",                            # LAB
        "Nominal Intermediate Input Compensation",               # II
        "Nominal IT Equipment Capital Compensation",             # vkit.
        "Nominal Non-college Labor Compensation",                # vln.
        "Nominal R&D Capital Compensation",                      # vkRD.
        "Nominal Other Capital Compensation",                    # vkoth.
        "Nominal College Labor Compensation",                    # vlcol.
        "Nominal Entertainment Originals Capital Compensation",  # vkart.
        "Nominal Software Capital Compensation",                 # vksoft.
        "Growth of Other Capital Quantity Index",                # dlog_qko.
        "Growth of IT Equipment Capital Quantity Index",         # dlog_qkit.
        "Growth of College Labor Quantity Index",                # dlog_qlindexcol_merge.
        "Growth of R&D Capital Quantity Index",                  # dlog_qkrd.
        "Growth of Gross Output Quantity Index",                 # dlog_goqi.
        "Growth of Non-college Labor Quantity Index",            # dlog_qlindexn_merge.
        "Growth of Entertainment Originals Quantity Index",      # dlog_qka.
        "Growth of Total Hours",                                 # dlog_hrs
        "Growth of Software Capital Quantity Index",             # dlog_qks.
        "Growth of Intermediate Input Quantity Index",           # dlog_iiqi.
        "Nominal Energy Intermediate Compensation",              # vien.
        "Nominal Materials Intermediate Compensation",           # vimt.
        "Nominal Services Intermediate Compensation",            # visv.
        "Growth of Services Intermediate Input Quantity Index",  # dlog_qisv.
        "Growth of Materials Intermediate Input Quantity Index", # dlog_qimt.
        "Growth of Energy Intermediate Input Quantity Index"     # dlog_qien.
    ]
})

#Export Excel with multiple sheets
os.makedirs(export_file_path, exist_ok=True)
output_file = os.path.join(export_file_path, "cleaned_data.xlsx")

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    #Sheet 1 -- Legend
    var_defs.to_excel(writer, sheet_name="Variable_Definitions", index=False)
    
    #Sheets 2/3 -- Data
    df_1963to2023.to_excel(writer, sheet_name="df_1963to2023", index=False)
    df_1947to2023.to_excel(writer, sheet_name="df_nom_1947to2023", index=False)